In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

import pandas as pd
import numpy as np

In [2]:
# source: https://github.com/huggingface/notebooks/blob/main/examples/protein_language_modeling.ipynb

In [2]:
TARGETSCAN_DATASET_PATH = "/sci/nosnap/michall/roeizucker/token_classification/targetscan_RNA_combined_nrows_90000_hg38.csv"

model_name = "InstaDeepAI/nucleotide-transformer-500m-1000g"


In [4]:
# model.config.features

In [5]:
# predicted_tokens_classes

In [6]:
df = pd.read_csv(TARGETSCAN_DATASET_PATH)

df["result"] = df["result"].str.replace("'","")
df["result"] = df["result"].apply(lambda x:x.strip('][').split(', '))
df["stags"] = df["stags"].apply(lambda x:x.strip('][').split(', '))
df['stags'] = df['stags'].apply(lambda x: list(map(int, x)))


In [7]:
# # working
# sequences = []
# labels = []

# for row_idx, row in df.iterrows():

#     curr_label = []
#     for i, seq in enumerate(row["result"]):
#         curr_label.extend([row["stags"][i]]*len(seq))
#     if len(curr_label) > 1000:
#         continue
#     if len(curr_label) %6 != 0:
#         label = np.zeros(len(curr_label)//6,dtype=np.int64)
#     else:
#         continue
        
#     for i in range(0,len(curr_label),6):
#         if i+6 <len(curr_label) and sum(curr_label[i:i+6]) > 1:
#             label[i//6:i//6+1] = 1
#         else: # unneeded
#             label[i//6:i//6+1] = 0
#     sequences.append( "".join(row["result"]))
#     labels.append(label)


In [8]:
# working
sequences = []
labels = []

for row_idx, row in df.iterrows():

    curr_label = []
    for i, seq in enumerate(row["result"]):
        curr_label.extend([row["stags"][i]]*len(seq))
    if len(curr_label) > 1000:
        curr_label = curr_label[:999] # TODO: change so even after the label it's added, split region to multiple spaces
    if len(curr_label) %6 != 0:
        label = np.zeros(len(curr_label)//6,dtype=np.int64)
    else:
        continue
        
    for i in range(0,len(curr_label),6):
        if i+6 <len(curr_label) and sum(curr_label[i:i+6]) > 1:
            label[i//6:i//6+1] = 1
        else: # unneeded
            label[i//6:i//6+1] = 0
    sequences.append( "".join(row["result"])[:len(curr_label)])
    labels.append(label)


In [9]:
from sklearn.model_selection import train_test_split
train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=False)

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
train_tokenized = tokenizer(train_sequences)
test_tokenized = tokenizer(test_sequences)

In [11]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_tokenized)
test_dataset = Dataset.from_dict(test_tokenized)

train_dataset = train_dataset.add_column("labels", train_labels)
test_dataset = test_dataset.add_column("labels", test_labels)

In [12]:
i = 9
print(sequences[i])
print(labels[i])
print(len(sequences[i]))
print(len(labels[i]))

TAATTCAAAGTCAGAAAGATTTACGCCATGTTTTCTTTGAAGATACTTACAATATTAGCTCTTACATTTAGATCTTAGGTCAACTTTGAGTTAATTTTTATATGCAATATGAGGTA
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
116
19


In [13]:
# to_remove = []
# for i,val in enumerate(train_dataset):
#     if ((len(val["input_ids"]) -len(val["labels"]))!= 2):
#         to_remove.append(i)
# # what we don't want
# exclude_idx = to_remove

# # create new dataset exluding those idx
# train_dataset = train_dataset.select(
#     (
#         i for i in range(len(train_dataset)) 
#         if i not in set(exclude_idx)
#     )
# )
# a = 1/0

In [14]:
# print(to_remove[:10])

In [15]:
# print(train_dataset[0])
# print(sequences[0])
# print(labels[0])
# # print(train_dataset[1])


In [16]:
to_remove = []
print(len(train_dataset))

for i,val in enumerate(train_dataset):
    if ((len(val["input_ids"]) -len(val["labels"]))!= 2):
        # if ((len(val["input_ids"]) -len(val["labels"]))!= 2) or val["labels"][-2] == 1 or val["labels"][-1] == 1:
        to_remove.append(i)
        # print(len(train_sequences[i]) % 6)
# what we don't want
exclude_idx = to_remove

# create new dataset exluding those idx
# train_dataset = train_dataset.select(
#     (
#         i for i in range(len(train_dataset)) 
#         if i not in set(exclude_idx)
#     )
# )
print(len(train_dataset))
# a = 1 / 0

7999
7999


In [17]:
to_remove[:20]

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23]

In [18]:

def do_print(i):
    print(sequences[i])
    print(labels[i])
    print(len(sequences[i]))
    print(len(labels[i]))
    print(len(sequences[i])/6)
    print(len(labels[i]))
    print(train_dataset[i])
    print(len(train_dataset[i]["input_ids"]))
    print("#"*30)
do_print(819)
do_print(820)
do_print(821)


CTCCTGTGTGGTGGGGCTTGCCAGCGAGGGAGCAGGGAACAGACTGGGTTTGGAAAACAGTGGTCATCATGGACCCATTTCTGATTTATAACAAACTCATTCCCAATAAAGATGAAGGCTGCAGTCCACCAGGGTTATAATTATAAATAAATATACATGCAGCTAAAAGAGGATATTAGGTGAAAACAACTGACCTAGCATCTGCCCAAAGAAGTCCCCATTTTGTTGTTCATTATCAAAGTCCTGAAAATGAACCCTAGGTTTTTGGCTAAGAAGGTGGCCTCAGACCAGCCAAGCTGACCTTGGCACTTGGCTGGCTTCTGTGAGGCAGTAGAGTGCCCACACATAAGCTCACCACCTGTGCCCACCTCCTCCCTTCTCTCCCATGCCACCCCACTTGCTTCCAAGGGCTTGGTTTCCAAAGTGACATCCAGGGTGTAAGAGGTTGGGGAAAACGTCCTGCAAGGTGGCTCAGGGATCTGATTCCATCAGATGGTCTCATGAATACTGTGGGAGATTAAATCCATCTCAAAATAGGCAACCAATGCTATATTCTGAATGTTAGGTCTCTGGACTGAGTCCCACCCACCCACCCACTCAGGAGTGGGTTCTGAATGGCCCTCAGACCTCTCCTGGTGAGTCTTAATGGGGTGGGAGGCTGTTAGTTAAAAAATAAACACCTCTTCAGTAGGCTGGGGCCCTTTCCTTTTCCTCCACTTGGGGGGAAAATTGTGCTTCCAACTTGCAGGAGAATGAGCCGGATCACCACTAGAGAGCCCGGCTGGGCCAGGGGGTCAGAGCCCTTTCCAGTGGGCCCCAGAGGCCTGGGGAGGGCAGGAGTTTGAGGCACACTGTTCTTTCTCCAGGATGGAAAGTGGACAGCTGGACATGAGGTGGGCTAGCTCTGCATGCTCCTGACTTAGGGAGAAATAACATTTTCTTTCCCTTTTTTGTGTTTAGAATAATTTCCAAAGCAGTGTGCTCAGGCCTCCCTCCTGC


In [19]:
to_remove = []
print(len(test_dataset))

for i,val in enumerate(test_dataset):
    if ((len(val["input_ids"]) -len(val["labels"]))!= 2):
    # if ((len(val["input_ids"]) -len(val["labels"]))!= 2) or val["labels"][-2] == 1 or val["labels"][-1] == 1:
        to_remove.append(i)
# what we don't want
exclude_idx = to_remove

# create new dataset exluding those idx
# test_dataset = test_dataset.select(
#     (
#         i for i in range(len(test_dataset)) 
#         if i not in set(exclude_idx)
#     )
# )
# print(len(test_dataset))

2667


In [20]:
# counter = 0
# for val in train_dataset:
#     print(counter,len(val["input_ids"]),len(val["labels"]))
#     counter+=1

In [21]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForTokenClassification.from_pretrained(model_name,num_labels=num_labels, device_map="auto")

# model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

2024-11-10 13:23:19.179870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 13:23:19.191042: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 13:23:19.194535: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 13:23:19.203520: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 13:23:24.398528: W tensorflow/compiler/tf2

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [23]:
batch_size = 1
# TODO: USE their parameters and LORA. copy from them
args = TrainingArguments(
    f"{model_name}-finetuned-secondary-structure",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.001,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

/sci/nosnap/michall/roeizucker/new_python_env/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
from evaluate import load
import numpy as np

metric = load("accuracy")

def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    labels = labels.reshape((-1,))
    predictions = np.argmax(predictions, axis=2)
    predictions = predictions.reshape((-1,))
    predictions = predictions[labels!=-100]
    labels = labels[labels!=-100]
    return metric.compute(predictions=predictions, references=labels)

In [25]:
# !export CUDA_LAUNCH_BLOCKING=1


In [26]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()

TrainOutput(global_step=23997, training_loss=0.15317582883412387, metrics={'train_runtime': 4953.2482, 'train_samples_per_second': 4.845, 'train_steps_per_second': 4.845, 'total_flos': 8025630739228872.0, 'train_loss': 0.15317582883412387, 'epoch': 3.0})

In [27]:
# !set PYTORCH_CUDA_ALLOC_CONF="expandable_segments:True"
# !echo $PYTORCH_CUDA_ALLOC_CONF
# !echo 123
# for i in range(len(train_dataset)):
#     print(i,len(train_dataset[i]["labels"]),len(train_dataset[i]["input_ids"]))

In [28]:
# for i in range(len(train_dataset)):
#     if len(train_dataset[i]["labels"]) == 3260:
#         print(i)
#         print(len(train_dataset[i]["input_ids"]))
#     # print(len(train_dataset[i]["input_ids"]))
#     # print(len(train_dataset[i]["labels"]))
#     # print(len(sequences[i]))

In [29]:
i = 5
print(len(sequences[i]) // 6)
print(len(labels[i]))

166
166


In [42]:
def show_for_num(num):
    predict_dataset = Dataset.from_dict(test_dataset[num:num+1])
    raw_pred, _, _ = trainer.predict(predict_dataset)
    y_pred = np.argmax(raw_pred, axis=2)
    print(y_pred[0])
    print(np.array(test_dataset[num]["labels"]))
    print("*******"*11)
show_for_num(9)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
*****************************************************************************


In [31]:
# y_pred[0]

In [32]:
np.array(test_dataset[2]["labels"])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [33]:
predictions 
predictions = np.argmax(predictions, axis=2)

NameError: name 'predictions' is not defined

In [ ]:
len(predict_dataset["input_ids"])

In [ ]:
len(predict_dataset[0]["input_ids"])

In [ ]:
predictions, labels, metrics = trainer.predict(predict_dataset, metric_key_prefix="predict")
metrics["predict_samples"] =  len(predict_dataset)

trainer.log_metrics("predict", metrics)
trainer.save_metrics("predict", metrics)

# predictions = np.argmax(predictions, axis=1)

In [ ]:



# # output_predict_file = os.path.join(training_args.output_dir, "predictions.txt")
# for index, item in enumerate(predictions):
#     # item = label_list[item]
#     print(f"{index}\t{item}\n")
